<a href="https://colab.research.google.com/github/YaserMarey/my_openai_colab/blob/master/semantically_searchable_video/semantically_searchable_arabic_language_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Semantically Searchable Arabic Language Youtube Video

Whisper is an automatic speech recognition (ASR) system developed by OpenAI. It has been trained on 680,000 hours of multilingual and multitasking supervised data collected from the web. This large and diverse dataset leads to improved robustness to accents, background noise, and technical language.

But how does it perform in Arabic?

according to [whisper](https://github.com/openai/whisper) "Whisper's performance varies widely depending on the language." with WER (Word Error Rate) of 15.7 for Arabic compared to 4.2 for English.

WER is calculated by comparing the transcribed text produced by the speech recognition system to a reference transcription and counting the number of substitutions, deletions, and insertions required to align the two texts. The WER is then calculated as the total number of errors divided by the total number of words in the reference transcription.

In this notebook, I will try to ad-hock test Whisper's performance in Arabic. 

I will apply Whisper to transcript a [short video](https://www.youtube.com/watch?v=RgG5e87-Lbs). The [video](https://www.youtube.com/watch?v=RgG5e87-Lbs) will be in Arabic with modern Egyptian Dialect which usually contains English words or sentences. I will then embed text segments and overlap them and then query these segments by calculating the cosine similarity.

Found segments will then point us to the place in the video relevant to the query.

#### Steps:
- Download the video from YouTube
- Transcript video using Whisper
- Combine segments for more meaningful chunks
- Text embed transcripted segments
- Text embed query
- Calculate similarity


##### First let's install dependencies

In [1]:
!pip install openai pytube tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 3.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [2]:
!pip install git+https://github.com/openai/whisper.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-24jthg9f
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-24jthg9f
  Resolved https://github.com/openai/whisper.git to commit 6dea21fd7f7253bfe450f1e2512a0fe47ee2d258
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 KB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796926 sha256=fc8c80e932a04d5198cfac0740e583530c90c343ba24fbdee4d6cd7a9536fc06
  Stored in directory: /tmp/pip-ephem-wheel-cache-2fl0pfsg/wheels/fe/03/29/e7919208d11b4ab329

##### Now download the video and extract the audio track.

In [3]:
import whisper
from pytube import YouTube
youtube_video_url = "https://www.youtube.com/watch?v=RgG5e87-Lbs"
youtube_video = YouTube(youtube_video_url)
audio = youtube_video.streams.get_audio_only().download()


#### load the smallest model ( base ) and transcript the extracted audio in the step above.

In [5]:
model = whisper.load_model('base')
transcript_output = model.transcribe(audio)
transcript_output

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 104MiB/s]


{'text': ' الزيتم تأسيري الاسبلط سكرين على دوبر من يرقرون يلّبين هذا أول حاجة نعملها ينجيب بيدي هنا لحنا نسردهم من حطهم فوقات ويزعمنا عاديك وبعدك ده نختار الفيديو اللي فوق نروح على فكت كنترولز ونلعب في البوزيشة اللغات مقابق في نصششة بالزلط حرى كده بالسة ونجيبوا لغائطه فبكنا كنت عسنة الشاشة نفسي بس في مشكلة إنك في جوزء من الفيديو نحت اللي منها نحفنا عايزين يصور عايزين نظر مسلم حيث تلعلي مندي في الحال مشكلة ده أنا جاهت أن يزعم كامت وبعدك ده نروح على فكت سيكون هناك ده نعمل السشة على فكت اسمه رب نحو ده الافكت ونعملوا دراجن رب والخطو على الفيديو اللي فوق ونقذو الحالت دي الفيديو اللي فوق نظر طالي ونروح على فكت قنترولز حلق إيمان جود هنئي في فكت قنترولز بالشكل ذكه اللي عندنا عايز ارباء فشنستحط بعض كده يوم اللي فتوطب رايد تبطه كروب يحن تترطى حتى من الفيديو فنته بيولى بعيد تطعم انه حتى من الشمال واللمين واللمين فوق اللي من بحطة فانقام الشمال شوية اللي قطم حسنوية يضظمقت يعني بالشكل ذكه وبعدك ده نطالفو نروح على البزيشن نوم رجعينه تنئي نصم طروح ونص البديو وبعدك ده بعد البعي ساح لمنكن نروحنا من سلكت

The quality of the transcript is not good at all, no need to even count WER.

Now let's try the rest of the models, Small, Medium, and Large.

In [6]:
model = whisper.load_model('small')
transcript_output = model.transcribe(audio)
transcript_output

100%|███████████████████████████████████████| 461M/461M [00:06<00:00, 78.0MiB/s]


{'text': ' إذا تفعل تأسير الهاتف لإشتراك على أدوة برميير برو يلو بنا أول حاجة نجيب فيديو نستخدمهم ونضعهم فيه وكما نفعل وبعد ذلك نختار الفيديو ونذهب إلى افكت كونترولز ونلعب في الوضع اللغات ما يبقى في نص الشاشة بالضبط حراك هذا بسهل نجيبه اللغاية بها يمكن أن نكون عسنة الشاشة نصي بس في مشكلة إنك في جزء من الفيديو نحيت الليمين ها لأنها خفنة عايزين ويظهر عايزين نظر مثل حتى الليمين دي فأي الحل المشكلة دي نرجحة نزي مكانك وبعد ذلك نذهب إلى افكت اللي هي هنا كذا نمسش على افكت اسمه روب ناخد الافكت ونعملوا دراجن روب ونخطوه على الفيديو اللي فوق ونركزوا الحلتة دي ونفوق نضط عليه ونروح الافكت كونترولز حلقي موجود هنا ايه الافكت كونترولز بشكل ذاك حلقي عندنا ايه اربع افشان سحت باعة كذا اليوم اللفت وطب ورايت وبوتو كروبي يعني انت ترطع حتى من الفيديو فانتر بيقولك بعاية ترطع من انو حتى من الشمال واللميمين والمن فوق والمن تحت فاناقع من الشمال شوية اللي اللي نحس من هويه اتضمط يعني بشكل ذاك وبعد كذا نطلع فوق نروح الافكت ونمرجعينه تانيه نسانطروه ونص الفيديو وبعد كذا بقى البعيسة لممكن نروح نعمل سلقت الفيديو التاني

Better, but still not acceptable.

In [7]:
model = whisper.load_model('medium')
transcript_output = model.transcribe(audio)
transcript_output

100%|█████████████████████████████████████| 1.42G/1.42G [00:18<00:00, 84.7MiB/s]


{'text': ' كيف تقوم بتأثير الـ Split Screen على Adobe Premiere Pro لنبدأ أولا سنقوم بإستخدام فيديو هنا ونضعه على بعض المقاطع كما قمنا بفعل ثم نختار الفيديو الأعلى ونذهب إلى Effect Controls ونلعب في مكان الغاية المتصل في نصف الشاشة نحرك هذا بالسهل ونضعه على الغاية هنا يمكننا أن نكون أسنانا الشاشة نصف ولكن هناك مشكلة هناك جزء من الفيديو الأعلى ونحن نريد أن يصوره نريد أن نصور مثلاً هذه الأعلى فماذا نقوم بإجراء هذه المشكلة؟ نعود إلى مكان الغاية المتصل ثم نذهب إلى Effects نضعه هنا نقوم بإستخدام Effect name Crop نأخذ Effect ونضعه على الفيديو الأعلى ونركز في هذا المكان نضعه على ونذهب إلى Effect Controls ونرى ماذا يوجد هنا في هذا المكان لدينا 4 أفتحة يوجد left, top, right, bottom Crop يعني أنك تقطع حتة من الفيديو فأنت تقول أن تقطع من أنا وحتة من الشمال أو من اليمين أو من فوق أو من تحت فنقطع من الشمال قليلاً لنحس أنه قطع حتة بشكل أكيد ثم نخرج إلى فوق نذهب إلى Position ونعود إلى مكان الثاني نقوم بمسطره في نصف الفيديو وبعد ذلك يمكننا أن نقوم بعمل Select الفيديو الثاني ونذهب إلى Position ونحركه لغ

Much better transcript this time.

In [8]:
model = whisper.load_model('large')
transcript_output = model.transcribe(audio)
transcript_output

100%|██████████████████████████████████████| 2.87G/2.87G [00:22<00:00, 134MiB/s]


{'text': ' كيفية تأثير الـ Split Screen على الـ Adobe Premiere Pro هيا بنا أول شيء سنفعله هو جلب فيديوهاتنا ونضعها فوق بعضنا كما نفعل وبعد ذلك نختار الفيديو الذي في الأعلى ونذهب إلى Effect Controls ونلعب في المقاومة حتى لا يبقى في نصف الشاشة بالضبط سنفعل هذا فقط ونجلبه إلى هنا يمكن أن نكون أثناء نقص الشاشة ولكن هناك مشكلة هناك جزء من الفيديو من ناحية اليمين نحن خافناه ونريده أن يظهر نريده أن يظهر على اليمين فما هي حل المشكلة؟ سأعود إلى المكان وبعد ذلك نذهب إلى Effects وهنا ونضع الشاشة على effect اسمه Drop نضعه على الفيديو الذي في الأعلى ونركز على الفيديو الذي في الأعلى نضغط عليه ونذهب إلى Effects Controls سنجد ماذا يوجد هنا بشكل هذا الآن لدينا أربع أفشل مرتبطة اليوم لفت وطوب ورايت وبوتن كروب يعني أنك تقطع حتى من الفيديو فأنت يريد أن تقطع من أي حتى من الشمال أو من اليمين أو من فوق أو من تحت سنقطع من الشمال قليلاً لكي نشعر أنه تم تصميمه بشكل هذا وبعد ذلك نذهب إلى الأعلى نذهب إلى Position ونعود إلى نسنتره في نصف الفيديو وبعد ذلك يصبح الأمر سهلاً نستطيع أن نقوم بمقاطع الفيديو الثانية ونذهب

Almost perfect, spotted only two errors, much lower WER compared to the official 15.7

Now that we have an acceptable WER transcript, let's preprocess the text segments, and first prepare the list of segments with links to their corresponding places in the video.

In [9]:
transcript_data = []
for segment in transcript_output['segments']:
        transcript_data.append({
                "id": f"{youtube_video_url}&t={segment['start']}s",
                "text": segment["text"].strip(),
                "start": segment['start'],
                "end": segment['end']
        })


In [11]:
transcript_data[0:5]

[{'id': 'https://www.youtube.com/watch?v=RgG5e87-Lbs&t=0.0s',
  'text': 'كيفية تأثير الـ Split Screen على الـ Adobe Premiere Pro',
  'start': 0.0,
  'end': 3.2},
 {'id': 'https://www.youtube.com/watch?v=RgG5e87-Lbs&t=3.48s',
  'text': 'هيا بنا',
  'start': 3.48,
  'end': 4.48},
 {'id': 'https://www.youtube.com/watch?v=RgG5e87-Lbs&t=11.24s',
  'text': 'أول شيء سنفعله هو جلب فيديوهاتنا ونضعها فوق بعضنا',
  'start': 11.24,
  'end': 14.84},
 {'id': 'https://www.youtube.com/watch?v=RgG5e87-Lbs&t=14.84s',
  'text': 'كما نفعل',
  'start': 14.84,
  'end': 16.080000000000002},
 {'id': 'https://www.youtube.com/watch?v=RgG5e87-Lbs&t=16.080000000000002s',
  'text': 'وبعد ذلك نختار الفيديو الذي في الأعلى',
  'start': 16.080000000000002,
  'end': 18.080000000000002}]

Combining each 5 (window) transcript segments into one, with overlapping of 2 segments =(stride), these numbers might need fine tunning to obtain the most meaningful segmentation

In [12]:
transcript_new_data = []
window = 5  # number of segments to combine
stride = 2  # number of segments to 'stride' over, used to create overlap
for i in (range(0, len(transcript_data), stride)):
    i_end = min(len(transcript_data)-1, i+window)
    text = ' '.join(_['text'] for _ in transcript_data[i:i_end])
    transcript_new_data.append({
        'start': transcript_data[i]['start'],
        'end': transcript_data[i_end]['end'],
        'text': text,
        'id': transcript_data[i]['id'],
    })


In [14]:
transcript_new_data[0:5]

[{'start': 0.0,
  'end': 20.080000000000002,
  'text': 'كيفية تأثير الـ Split Screen على الـ Adobe Premiere Pro هيا بنا أول شيء سنفعله هو جلب فيديوهاتنا ونضعها فوق بعضنا كما نفعل وبعد ذلك نختار الفيديو الذي في الأعلى',
  'id': 'https://www.youtube.com/watch?v=RgG5e87-Lbs&t=0.0s'},
 {'start': 11.24,
  'end': 24.76,
  'text': 'أول شيء سنفعله هو جلب فيديوهاتنا ونضعها فوق بعضنا كما نفعل وبعد ذلك نختار الفيديو الذي في الأعلى ونذهب إلى Effect Controls ونلعب في المقاومة حتى لا يبقى في نصف الشاشة بالضبط',
  'id': 'https://www.youtube.com/watch?v=RgG5e87-Lbs&t=11.24s'},
 {'start': 16.080000000000002,
  'end': 27.44,
  'text': 'وبعد ذلك نختار الفيديو الذي في الأعلى ونذهب إلى Effect Controls ونلعب في المقاومة حتى لا يبقى في نصف الشاشة بالضبط سنفعل هذا فقط ونجلبه',
  'id': 'https://www.youtube.com/watch?v=RgG5e87-Lbs&t=16.080000000000002s'},
 {'start': 20.76,
  'end': 31.32,
  'text': 'ونلعب في المقاومة حتى لا يبقى في نصف الشاشة بالضبط سنفعل هذا فقط ونجلبه إلى هنا يمكن أن نكون أثناء نقص الشاشة',
 

Now, we text embed all segments

In [16]:
# imports
import pandas as pd
import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity


In [17]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191


In [18]:
encoding = tiktoken.get_encoding("cl100k_base")
# should print [83, 1609, 5963, 374, 2294, 0]
encoding.encode("tiktoken is great!")

[83, 1609, 5963, 374, 2294, 0]

In [ ]:
import openai
df = pd.DataFrame(transcript_new_data)
openai.api_key = 'sk-PUT YOUR OPENAI API KEY'
df["embedding"] = df.text.apply(lambda x: get_embedding(x, engine=embedding_model))
df[0:5]

start    end                                               text  \
0   0.00  20.08  كيفية تأثير الـ Split Screen على الـ Adobe Pre...   
1  11.24  24.76  أول شيء سنفعله هو جلب فيديوهاتنا ونضعها فوق بع...   
2  16.08  27.44  وبعد ذلك نختار الفيديو الذي في الأعلى ونذهب إل...   
3  20.76  31.32  ونلعب في المقاومة حتى لا يبقى في نصف الشاشة با...   
4  25.16  35.16  ونجلبه إلى هنا يمكن أن نكون أثناء نقص الشاشة و...   

                                                  id  n_tokens  \
0  https://www.youtube.com/watch?v=RgG5e87-Lbs&t=...        97   
1  https://www.youtube.com/watch?v=RgG5e87-Lbs&t=...       114   
2  https://www.youtube.com/watch?v=RgG5e87-Lbs&t=...        86   
3  https://www.youtube.com/watch?v=RgG5e87-Lbs&t=...        79   
4  https://www.youtube.com/watch?v=RgG5e87-Lbs&t=...        70   

                                           embedding  
0  [-0.0026962889824062586, -0.001564752776175737...  
1  [-0.022274624556303024, -0.007607742678374052,...  
2  [-0.0182399470359087, -0.009821509011089802, -...  
3  [-0.013533576391637325, -0.014525429345667362,...  
4  [-0.01303117722272873, -0.023809239268302917, ...

We then embed the query, here the question is when in the video the YouTuber asked us to subscribe to the channel.


In [22]:
query = 'امتى طلب اننا نعمل إشتراك فى القناه ؟'
query_embedding = get_embedding(query, engine=embedding_model)


We find the most relevant parts of the video transcript to the query

In [23]:
df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding))
results = (df.sort_values("similarity", ascending=False).head(3))
results = results.set_index(['id'])

In [24]:
from IPython.display import HTML
HTML(results[['start','similarity', 'text']].to_html(render_links=True, escape=False))

References: https://lablab.ai/t/whisper-transcribe-youtube-video